# Goal
1. genearate data set as CNN network input


# Process

In [ ]:
import numpy as np
#获得４＊Ｌ维０１矩阵的函数
d = {'a':0, 'A':0, 'g':1, 'G':1, 'c':2, 'C':2, 't':3, 'T':3, 'N':4, 'n':4}
def seq_to_mat(seq):
    mat = np.zeros((5,len(seq)))
    for i in range(len(seq)):
        mat[d[seq[i]],i] =1
        #mat[d[seq[i]],i] =1
        #mat[d[seq[i]],i] = 1
        pass
    mat = mat[:4,:]
    return mat

In [ ]:
from collections import defaultdict
seq_freq = defaultdict(int)

In [ ]:
rootdir =  '/home/yinqijin/WorkSpace/2.RNA_Structure_Profile'
cutdown_len  = 13

data = []
with open(rootdir+'/Orig_data/sharpr.txt','r')  as f:
    with open(rootdir+'/Orig_data/score.txt','r') as f1:     
        while 1:
            name = f.readline()
            if not name :
                break
            sequence = f.readline()
            scores = f1.readline().split()[1:]  #得到每个碱基的分数
            bp_name  = name.split('_')  [:-1]
            bp_point = name.split('_')[-1]
            for bp_iter  in range(0,len(sequence)-cutdown_len-1):
                bp_seq_temp = {}
                bp_seq_temp['seq'] =sequence[bp_iter:(bp_iter+cutdown_len)].strip()
                #seq_temp= bp_seq_temp['seq']
                bp_seq_temp['mat'] = seq_to_mat(bp_seq_temp['seq'] )
                bp_seq_temp['name'] = bp_name[0]+'_'+bp_name[1]+'_'+bp_name[2]+'_'+bp_name[3]+'_'+ str(int(bp_point)+bp_iter*(cutdown_len-1))
                scores_str = scores[bp_iter:(bp_iter+cutdown_len)]
                bp_seq_temp['score'] = [float(item) for item in scores_str]
                data.append(bp_seq_temp)
                `
                
        
        
     

In [ ]:

from collections import defaultdict
d = defaultdict(list)
d ['aa'] .append(1)
d['aa'].append(2)
print d['aa']

In [ ]:
print  data[1],data[282]

In [ ]:
print seq_to_mat(data[283]['seq'])

# Change Format

In [ ]:
data2 = {}
for key in data[0].keys():
    data2['seq'] = [item['seq'] for item in data]
    data2['name'] = [item['name'] for item in data]
    data2['score'] = [item['score'] for item in data]
    data2['mat'] = [item['mat'] for item in data]

In [ ]:
print data2['seq'][:2]
print data2['name'][:2]
print data2['score'][:2]
print data2['mat'][:2]

In [ ]:
#统计seq出现的次数

from collections import defaultdict
#rom collections import Counter


seqdict_order = defaultdict(list)
#seqdict = Counter(data2['seq'])
for index in range(0,len(data2['seq'])):

    seqdict_order[ data2['seq'][index]].append(int(index))

remove_point = []
for key in seqdict_order.keys():
    key_dict = seqdict_order[key]
    if 1 !=  len(key_dict):
        for key_order in key_dict:
            remove_point.append(key_order)
remove_point  = sorted(remove_point)

print remove_point[:4]

In [ ]:
len(remove_point)

In [ ]:
slicea = slice (1,4)
print data2['seq'][slicea]

In [ ]:
data_2_prun = {}
slicea  =slice([1,2,3])
for key in data2.keys:
    data_2_prun[key] = data2[key]

In [ ]:
low =seqdict.most_common(500)
print low[:2]


# Save Data

In [ ]:
import hickle  as hkl

In [ ]:
hkl.dump(data2,'./Gen_data/CROSS_Net_input_0.hkl')

### Tips  删除u

# Generate NetWork

In [ ]:
import theano
from lasagne.layers import InputLayer
from lasagne.layers import DenseLayer
from lasagne.layers import DropoutLayer
try:
    from lasagne.layers.dnn import Conv2DDNNLayer as Conv2DLayer
    from lasagne.layers.dnn import MaxPool2DDNNLayer as MaxPool2DLayer
    print 'Using Lasagne.layers.dnn (faster)'
except ImportError:
    from lasagne.layers import Conv2DLayer
    from lasagne.layers import MaxPool2DLayer
    print 'Using Lasagne.layers (slower)'
from lasagne.nonlinearities import softmax, rectify, leaky_rectify
from lasagne.updates import adam
from lasagne.objectives import squared_error

from nolearn.lasagne import NeuralNet
from nolearn.lasagne import TrainSplit
from nolearn.lasagne import objective
from nolearn.lasagne import BatchIterator

from lasagne.nonlinearities import tanh 
floatX = theano.config.floatX

In [ ]:
cutdown_len = 13
layer1 = InputLayer(shape=(None,1,cutdown_len,4))
layer2 = DenseLayer(layer1, num_units=20 ,nonlinearity=tanh)
network = DenseLayer(layer2,num_units=1,nonlinearity=tanh)

In [ ]:

lr = theano.shared(np.float32(1e-4))

net = NeuralNet(
            network,
            max_epochs=7,
            update=adam,
            update_learning_rate=lr,
            train_split=TrainSplit(eval_size=0.1),
                regression = True,
            objective_loss_function = squared_error,
            #on_epoch_finished=[
            #    AdjustVariable(lr, target=1e-8, half_life=20)],
            verbose=4)

# Import Data

In [ ]:
X = data2['mat']
Y = data2['score']

In [ ]:
Y_ = [ item[(len(item)-1)/2] for item in Y]

In [ ]:
print len(X),len(Y_)
print Y_[:2]

In [ ]:
X_ = np.zeros((len(X),4,cutdown_len))
print X_.shape

In [ ]:
print  np.shape( X[284]),np.shape(X_[i,:,:])

In [ ]:
X_[:,:,:] = X[:]

for i in range(len(Y)):
    if i%10000==0 :
        print i
    X_[i,:,:] = X[i]

In [ ]:
Y_ = np.array(Y_)

In [ ]:
from sklearn.cross_validation import ShuffleSplit
rs = ShuffleSplit(len(Y_), n_iter=1)

In [ ]:
X_[:2]

In [ ]:
for train_idx, test_idx in rs:
    X_train = X_[train_idx,:]
    y_train = Y_[train_idx]
    X_test = X_[test_idx,:]
    y_test = Y_[test_idx]

In [ ]:
print X_train[:2]
print y_train[:2]

In [ ]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [ ]:
print X_train[:2]
print y_train[:2]

In [ ]:
cutdown_len = 13
layer1 = InputLayer(shape=(None,4,cutdown_len))
layer2 = DenseLayer(layer1, num_units=20 ,nonlinearity=tanh)
network = DenseLayer(layer2,num_units=1,nonlinearity=tanh)

lr = theano.shared(np.float32(1e-4))

net = NeuralNet(
            network,
            max_epochs=100,
            update=adam,
            update_learning_rate=lr,
            train_split=TrainSplit(eval_size=0.1),
                regression = True,
            objective_loss_function = squared_error,
            #on_epoch_finished=[
            #    AdjustVariable(lr, target=1e-8, half_life=20)],
            verbose=4)

In [ ]:
net.fit(X_train, y_train)

# Save network values

In [ ]:
from nolearn.lasagne import NeuralNet
net.initialize_layers()
net_params=[]
net.save_params_to('./Gen_data/CROSS_net_values.pkl')

# Test Network

## Load Network Values

In [ ]:
import theano
from lasagne.layers import InputLayer
from lasagne.layers import DenseLayer
from lasagne.layers import DropoutLayer
try:
    from lasagne.layers.dnn import Conv2DDNNLayer as Conv2DLayer
    from lasagne.layers.dnn import MaxPool2DDNNLayer as MaxPool2DLayer
    print 'Using Lasagne.layers.dnn (faster)'
except ImportError:
    from lasagne.layers import Conv2DLayer
    from lasagne.layers import MaxPool2DLayer
    print 'Using Lasagne.layers (slower)'
from lasagne.nonlinearities import softmax, rectify, leaky_rectify
from lasagne.updates import adam
from lasagne.objectives import squared_error

from nolearn.lasagne import NeuralNet
from nolearn.lasagne import TrainSplit
from nolearn.lasagne import objective
from nolearn.lasagne import BatchIterator

from lasagne.nonlinearities import tanh 
floatX = theano.config.floatX

cutdown_len = 13
layer1 = InputLayer(shape=(None,cutdown_len,4))
layer2 = DenseLayer(layer1, num_units=20 ,nonlinearity=tanh)
network = DenseLayer(layer2,num_units=1,nonlinearity=tanh)

lr = theano.shared(np.float32(1e-4))

net = NeuralNet(
            network,
            max_epochs=7,
            update=adam,
            update_learning_rate=lr,
            train_split=TrainSplit(eval_size=0.1),
                regression = True,
            objective_loss_function = squared_error,
            #on_epoch_finished=[
            #    AdjustVariable(lr, target=1e-8, half_life=20)],
            verbose=4)

In [ ]:
# Load  NetWork Values
from nolearn.lasagne import NeuralNet
net.initialize_layers()
net.load_params_from('./Gen_data/CROSS_net_values.pkl')

## Test

In [ ]:
y_1 = net.predict(X_test[:2])
print y_1,y_test[:2]

In [ ]:
y_pred = net.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.scatter(y_test,y_pred,0.1)

In [ ]:
from scipy import stats

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(y_test,y_pred[:,0])
print r_value